# Agent Starter Pack Setup Notebook

This notebook will help you set up your environment for working with the Agent Starter Pack. It will check for required dependencies and install any missing components.

## Prerequisites

This notebook will check for and help install:
- Python 3.10+
- Google Cloud SDK
- Terraform
- uv package manager
- GitHub CLI (optional, for CI/CD setup)
- Authentication requirements

Let's start by checking your current environment.

### starter pack Dir

```bash # */agent-starter-pack
sudo apt update && sudo apt install python3-full 
sudo apt install python3-pip
make install


#if using pipx
python3 -m pip install --user pipx && python3 -m pipx ensurepath
pipx install agent-starter-pack


# using venv
python3 -m venv venv \
source venv/bin/activate \
pip install agent-starter-pack
```

### Agent Dir

```bash
cd {project_name} # */agent-starter-pack/project_name
sudo add-apt-repository ppa:deadsnakes/ppa
sudo apt update

sudo apt install python3.10-full
sudo apt install chromium # or browser sudo apt install chromium-browser

make install

# using venv
python3.10 -m venv .venv
source .venv/bin/activate
make install

```

```bash

## 1. Check Python Environment

In [ ]:
import sys
import platform
import subprocess
import shutil
import os
from pathlib import Path

# Check Python version
python_version = sys.version_info
required_python_version = (3, 10)

print(f"Python version: {sys.version}")
print(f"Platform: {platform.platform()}")

if python_version < required_python_version:
    print(f"⚠️ Warning: Python version {python_version.major}.{python_version.minor} is below the required version 3.10+")
    print("Please consider upgrading your Python version before continuing.")
else:
    print(f"✅ Python version {python_version.major}.{python_version.minor} meets requirements")

## 2. Check for and Install Required Tools

Let's check for the required tools: uv, Google Cloud SDK, Terraform, and GitHub CLI.

In [ ]:
def check_command(command, args=["-V"], name=None):
    """Check if a command is available and return its version"""
    if name is None:
        name = command
        
    cmd_path = shutil.which(command)
    if cmd_path:
        try:
            result = subprocess.run([command] + args, capture_output=True, text=True)
            version = result.stdout.strip() or result.stderr.strip()
            print(f"✅ {name} is installed: {version}")
            return True, cmd_path, version
        except Exception as e:
            print(f"⚠️ {name} is installed at {cmd_path}, but encountered an error: {e}")
            return True, cmd_path, None
    else:
        print(f"❌ {name} is not installed or not in PATH")
        return False, None, None

# Check for uv
uv_installed, uv_path, uv_version = check_command("uv", ["--version"])

# Check for Google Cloud SDK
gcloud_installed, gcloud_path, gcloud_version = check_command("gcloud", ["--version"])

# Check for Terraform
terraform_installed, terraform_path, terraform_version = check_command("terraform", ["--version"])

# Check for GitHub CLI
gh_installed, gh_path, gh_version = check_command("gh", ["--version"])

# Check for make
make_installed, make_path, make_version = check_command("make", ["--version"])

## 3. Install Missing Dependencies

Based on the checks above, let's install any missing dependencies. You can run the cells for tools that are missing.

### Install uv (Python package manager)

uv is a fast package installer and resolver that we'll use for managing Python dependencies.

In [ ]:
if not uv_installed:
    print("Installing uv...")
    
    # Determine the installation command based on the platform
    if platform.system() == "Windows":
        # For Windows, download the installer and run it
        print("For Windows, please follow the installation instructions at:")
        print("https://astral.sh/uv/0.6.11/install.ps1")
    else:
        # For Unix-based systems (Linux and macOS)
        try:
            install_command = "curl -LsSf https://astral.sh/uv/install.sh | sh"
            print(f"Running: {install_command}")
            subprocess.run(install_command, shell=True, check=True)
            print("✅ uv installed successfully")
            
            # Remind users to update their PATH
            print("\nIMPORTANT: You may need to restart your terminal or add uv to your PATH.")
            print("If you're using bash, you can run: source ~/.bashrc")
            print("If you're using zsh, you can run: source ~/.zshrc")
        except subprocess.CalledProcessError as e:
            print(f"❌ Failed to install uv: {e}")
else:
    print("uv is already installed. Skipping installation.")

### Install Google Cloud SDK

The Google Cloud SDK is required for interacting with Google Cloud services.

In [ ]:
if not gcloud_installed:
    print("Google Cloud SDK is not installed. Please install it manually:")
    
    if platform.system() == "Windows":
        print("Windows: https://cloud.google.com/sdk/docs/install-sdk#windows")
    elif platform.system() == "Darwin":  # macOS
        print("macOS: https://cloud.google.com/sdk/docs/install-sdk#mac")
    else:  # Linux
        print("Linux: https://cloud.google.com/sdk/docs/install-sdk#deb")
        
    print("\nAfter installing, run 'gcloud init' to initialize the SDK.")
else:
    print("Google Cloud SDK is already installed.")

### Install Terraform

Terraform is used for infrastructure deployment.

In [ ]:
if not terraform_installed:
    print("Terraform is not installed. Please install it manually:")
    print("Instructions: https://developer.hashicorp.com/terraform/downloads")
    
    if platform.system() == "Darwin" and platform.machine() == "arm64":  # macOS Apple Silicon
        print("\nFor macOS with Apple Silicon, you can use Homebrew:")
        print("brew tap hashicorp/tap")
        print("brew install hashicorp/tap/terraform")
    elif platform.system() == "Darwin":  # macOS Intel
        print("\nFor macOS with Intel, you can use Homebrew:")
        print("brew tap hashicorp/tap")
        print("brew install hashicorp/tap/terraform")
    elif platform.system() == "Linux":
        print("\nFor Linux, you can use apt (Ubuntu/Debian):")
        print("sudo apt-get update && sudo apt-get install -y gnupg software-properties-common")
        print("wget -O- https://apt.releases.hashicorp.com/gpg | gpg --dearmor | sudo tee /usr/share/keyrings/hashicorp-archive-keyring.gpg")
        print("echo \"deb [signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] https://apt.releases.hashicorp.com $(lsb_release -cs) main\" | sudo tee /etc/apt/sources.list.d/hashicorp.list")
        print("sudo apt-get update && sudo apt-get install terraform")
else:
    print("Terraform is already installed.")

### Install GitHub CLI (optional)

The GitHub CLI is needed if you plan to use the `setup-cicd` command for automated CI/CD setup.

In [ ]:
if not gh_installed:
    print("GitHub CLI is not installed. It's optional but recommended for CI/CD setup.")
    print("Installation instructions: https://github.com/cli/cli#installation")
    
    if platform.system() == "Darwin":  # macOS
        print("\nFor macOS, you can use Homebrew:")
        print("brew install gh")
    elif platform.system() == "Linux":
        print("\nFor Ubuntu/Debian Linux:")
        print("""# First, remove any existing GitHub CLI repository configurations
sudo rm -f /etc/apt/sources.list.d/github-cli.list
sudo rm -f /etc/apt/keyrings/githubcli-archive-keyring.gpg

# Then add the repository with explicit settings
sudo mkdir -p -m 755 /etc/apt/keyrings
wget -qO- https://cli.github.com/packages/githubcli-archive-keyring.gpg | sudo tee /etc/apt/keyrings/githubcli-archive-keyring.gpg > /dev/null
sudo chmod go+r /etc/apt/keyrings/githubcli-archive-keyring.gpg
echo "deb [arch=$(dpkg --print-architecture) signed-by=/etc/apt/keyrings/githubcli-archive-keyring.gpg] https://cli.github.com/packages stable main" | sudo tee /etc/apt/sources.list.d/github-cli.list > /dev/null

# Update and install
sudo apt update
sudo apt install gh -y""")
        
        # You might want to add other distributions as well
        print("\nFor other Linux distributions, please refer to: https://github.com/cli/cli/blob/trunk/docs/install_linux.md")
else:
    print("GitHub CLI is already installed.")

### Install Make

Make is needed for running the project's predefined commands.

In [ ]:
if not make_installed:
    print("Make is not installed.")
    
    if platform.system() == "Windows":
        print("For Windows, you can install make through chocolatey:")
        print("choco install make")
        print("Or through scoop: scoop install make")
    elif platform.system() == "Darwin":  # macOS
        print("For macOS, you can use Homebrew:")
        print("brew install make")
    elif platform.system() == "Linux":
        print("For Ubuntu/Debian Linux:")
        print("sudo apt-get update && sudo apt-get install -y make")
else:
    print("Make is already installed.")

## 4. Verify Google Cloud Authentication

Let's verify that you're authenticated with Google Cloud.

In [ ]:
if gcloud_installed:
    try:
        # Check if gcloud is authenticated
        auth_result = subprocess.run(["gcloud", "auth", "list"], capture_output=True, text=True)
        if "No credentialed accounts." in auth_result.stdout:
            print("❌ Not authenticated with Google Cloud. Please run 'gcloud auth login' to authenticate.")
        else:
            print("✅ Authenticated with Google Cloud:") 
            print(auth_result.stdout)
            
        # Check if a project is set
        project_result = subprocess.run(["gcloud", "config", "get-value", "project"], capture_output=True, text=True)
        if project_result.stdout.strip():
            print(f"✅ Default project is set to: {project_result.stdout.strip()}")
        else:
            print("❌ No default project is set. Use 'gcloud config set project YOUR-PROJECT-ID' to set one.")
        
        # Check if application default credentials are set
        adc_file = Path.home() / ".config" / "gcloud" / "application_default_credentials.json"
        if adc_file.exists():
            print("✅ Application Default Credentials are set.")
        else:
            print("❌ Application Default Credentials are not set.")
            print("Run 'gcloud auth application-default login' to set them up.")
            print("This is required for using the Vertex AI API and other Google Cloud services.")
    except Exception as e:
        print(f"❌ Error checking Google Cloud authentication: {e}")
else:
    print("Google Cloud SDK is not installed. Authentication check skipped.")

### Authenticate with Google Cloud (if needed)

If you're not authenticated, run the following cells to authenticate.

In [ ]:
# Run this cell if you need to authenticate with Google Cloud
if gcloud_installed:
    try:
        print("Initiating Google Cloud authentication...")
        print("This will open a browser window to authenticate with Google Cloud.")
        subprocess.run(["gcloud", "auth", "login"], check=True)
        print("✅ Google Cloud authentication completed!")
    except Exception as e:
        print(f"❌ Error authenticating with Google Cloud: {e}")
else:
    print("Google Cloud SDK is not installed. Authentication skipped.")

In [ ]:
# Run this cell to set up Application Default Credentials (for Vertex AI and other APIs)
if gcloud_installed:
    try:
        print("Setting up Application Default Credentials...")
        print("This will open a browser window to authorize access to Google Cloud APIs.")
        subprocess.run(["gcloud", "auth", "application-default", "login"], check=True)
        print("✅ Application Default Credentials set up successfully!")
    except Exception as e:
        print(f"❌ Error setting up Application Default Credentials: {e}")
else:
    print("Google Cloud SDK is not installed. ADC setup skipped.")

## 5. Verify GitHub Authentication (Optional)

If you plan to use the CI/CD setup functionality, let's verify that you're authenticated with GitHub.

In [ ]:
if gh_installed:
    try:
        # Check if gh is authenticated
        auth_result = subprocess.run(["gh", "auth", "status"], capture_output=True, text=True)
        if "Logged in to" in auth_result.stdout:
            print("✅ Authenticated with GitHub:")
            print(auth_result.stdout.strip())
        else:
            print("❌ Not authenticated with GitHub. Please run 'gh auth login' to authenticate.")
    except Exception as e:
        print(f"❌ Error checking GitHub authentication: {e}")
else:
    print("GitHub CLI is not installed. Authentication check skipped.")

In [ ]:
# Run this cell if you need to authenticate with GitHub
if gh_installed:
    try:
        print("Initiating GitHub authentication...")
        print("This will guide you through the GitHub authentication process.")
        subprocess.run(["gh", "auth", "login"], check=True)
        print("✅ GitHub authentication completed!")
    except Exception as e:
        print(f"❌ Error authenticating with GitHub: {e}")
else:
    print("GitHub CLI is not installed. Authentication skipped.")

## 6. Set Up Python Virtual Environment and Install Dependencies

Let's set up a Python virtual environment and install the project dependencies.

In [ ]:
# Debug information about current location
print(f"Current working directory: {os.getcwd()}")
print(f"Directory contents: {os.listdir('.')}")

# Check if we're in a project directory created by agent-starter-pack
try:
    # Check for pyproject.toml in current directory
    pyproject_exists = os.path.exists("pyproject.toml")
    
    # Check for other key files/directories that would be in a project created by agent-starter-pack
    has_app_dir = os.path.exists("app")
    has_deployment_dir = os.path.exists("deployment")
    
    # Get parent directory name to verify we're within agent-starter-pack structure
    parent_dir = os.path.dirname(os.getcwd())
    parent_name = os.path.basename(parent_dir)
    
    print(f"Parent directory: {parent_dir}")
    print(f"Parent directory name: {parent_name}")
    
    if pyproject_exists:
        print("✅ Found pyproject.toml in the current directory.")
        
        if parent_name == "agent-starter-pack":
            print("✅ Parent directory is 'agent-starter-pack'. We're in the correct structure.")
        else:
            print("⚠️ Parent directory is not 'agent-starter-pack'. We might not be in the expected structure.")
            
        if has_app_dir and has_deployment_dir:
            print("✅ Found expected project directories (app/, deployment/).")
        else:
            missing = []
            if not has_app_dir: missing.append("app/")
            if not has_deployment_dir: missing.append("deployment/")
            print(f"⚠️ Missing some expected project directories: {', '.join(missing)}")
    else:
        print("❌ pyproject.toml not found in the current directory.")
        print("Please navigate to the project directory before continuing.")
except Exception as e:
    print(f"❌ Error checking for project structure: {e}")

# Check if virtual environment exists
venv_path = ".venv"
venv_exists = os.path.exists(venv_path) and os.path.isdir(venv_path)
if venv_exists:
    print(f"✅ Virtual environment exists at {venv_path}")
    # Print environment details
    try:
        activate_path = os.path.join(venv_path, "bin", "activate") if os.name != "nt" else os.path.join(venv_path, "Scripts", "activate")
        python_path = os.path.join(venv_path, "bin", "python") if os.name != "nt" else os.path.join(venv_path, "Scripts", "python.exe")
        
        print(f"Python interpreter path: {python_path}")
        print(f"Activation script: {activate_path}")
        
        if os.path.exists(python_path):
            # Get Python version from virtual environment
            result = subprocess.run([python_path, "--version"], capture_output=True, text=True)
            print(f"Virtual environment Python version: {result.stdout.strip()}")
    except Exception as e:
        print(f"Error getting virtual environment details: {e}")
else:
    print(f"❌ Virtual environment not found at {venv_path}")

In [ ]:
# Create virtual environment if it doesn't exist
if not venv_exists and pyproject_exists:
    try:
        print("Creating virtual environment...")
        subprocess.run([sys.executable, "-m", "venv", venv_path], check=True)
        print(f"✅ Virtual environment created at {venv_path}")
    except Exception as e:
        print(f"❌ Error creating virtual environment: {e}")

In [ ]:
# Install dependencies using make install (which uses uv)
if pyproject_exists and uv_installed:
    try:
        print("Installing dependencies using 'make install'...")
        subprocess.run(["make", "install"], check=True)
        print("✅ Dependencies installed successfully!")
    except Exception as e:
        print(f"❌ Error installing dependencies: {e}")
        print("\nAlternatively, you can try installing dependencies manually with:")
        print("uv sync --dev --extra streamlit --extra jupyter --frozen")
elif pyproject_exists and not uv_installed:
    print("❌ uv is not installed. Please install uv first and then run 'make install'.")
else:
    print("❌ Not in project directory. Please navigate to the project directory before continuing.")

## 7. Enable Required Google Cloud APIs

Let's enable the required Google Cloud APIs for the project. Make sure your default project is set correctly before running this cell.

In [ ]:
# Get the current project
if gcloud_installed:
    try:
        project_result = subprocess.run(["gcloud", "config", "get-value", "project"], capture_output=True, text=True)
        project_id = project_result.stdout.strip()
        if project_id:
            print(f"Current project: {project_id}")
            
            # Ask for confirmation
            user_input = input(f"Do you want to enable required APIs in project '{project_id}'? (y/n): ")
            if user_input.lower() == 'y':
                # Enable required APIs
                apis = [
                    "aiplatform.googleapis.com",       # Vertex AI
                    "discoveryengine.googleapis.com",  # Vertex AI Search
                    "cloudresourcemanager.googleapis.com",  # Resource Manager
                    "serviceusage.googleapis.com",     # Service Usage
                    "run.googleapis.com",              # Cloud Run
                    "logging.googleapis.com",          # Cloud Logging
                    "cloudtrace.googleapis.com"        # Cloud Trace
                ]
                
                print("Enabling required APIs...")
                for api in apis:
                    print(f"Enabling {api}...")
                    subprocess.run(["gcloud", "services", "enable", api], check=True)
                
                print("✅ All required APIs enabled successfully!")
            else:
                print("API enabling skipped.")
        else:
            print("❌ No default project is set. Use 'gcloud config set project YOUR-PROJECT-ID' to set one.")
    except Exception as e:
        print(f"❌ Error enabling APIs: {e}")
else:
    print("Google Cloud SDK is not installed. API enabling skipped.")

## 8. Start the Application

Now that everything is set up, let's start the application in playground mode.

In [ ]:
# Start the application in playground mode
if pyproject_exists and make_installed:
    print("Starting the application in playground mode...")
    print("This will open a local development environment with a backend and frontend.")
    print("The application will be available at http://localhost:8000 (backend) and http://localhost:8501 (frontend).")
    print("Press Ctrl+C to stop the application.")
    
    # Note that we're not actually starting the application here, just providing instructions
    print("\nTo start the application, open a terminal and run:")
    print("make playground")
    
    # Provide alternative instructions for starting backend/frontend separately
    print("\nAlternatively, you can start the backend and frontend separately:")
    print("Terminal 1: make backend")
    print("Terminal 2: make ui")
else:
    print("❌ Not in project directory or 'make' is not installed.")

## Setup Complete!

🎉 Congratulations! Your environment is now set up for working with the Agent Starter Pack.

### Next Steps

1. Start the application:
   ```
   make playground
   ```

2. Explore the Agent Starter Pack documentation:
   - [README.md](README.md)
   - [docs/README.md](docs/README.md)

3. Deploy your application to Google Cloud:
   - For development: `make setup-dev-env`
   - For production: See [deployment/README.md](deployment/README.md)

Happy coding!